In [ ]:
# default_exp export

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Graph module

> Provides main class and helpers classes and functions to handle pangenome graph and related data structures

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
notebook2script()

Converted 00_init.ipynb.
Converted 01_graph.ipynb.
Converted 02_tree.ipynb.
Converted 03_synteny.ipynb.
Converted 04_utils.ipynb.
Converted 05_export.ipynb.
Converted index.ipynb.


In [ ]:
#exporti

import os
import glob
import json
import time
import itertools
import warnings

import pdb

from copy import deepcopy

# import networkx as nx

# import pandas as pd
import numpy as np

from pangraph_constructor.utils import pathConvert,NpEncoder,adjustZoomLevels
from pangraph_constructor.graph import GenomeGraph

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
#export
componentTemplate = {
    "first_bin": 1,
    "last_bin": 1,
    "firstCol": 1,
    "lastCol": 1,
    "occupants": [],
    "matrix": [],
    "larrivals": [],
    "rarrivals": [],
    "ldepartures": [],
    "rdepartures": [],
    "x": 0
}

chunkTemplate = {
    "json_version":19,
    "first_bin":1,
    "last_bin": 1,
    "includes_connectors": True,
    "components": []
}

rootStructTemplate = {
    "json_version": 19,
    "pangenome_length": 0,#pangenomeLength, #total number of bp in pangenome
    "includes_connectors": True,
    "zoom_levels": {}
}


In [ ]:
#export
def exportToPantograph(graph=None,inputPath=None,GenomeGraphParams={},outputPath=None,outputName=None,outputSuffix=None,isSeq=True,
                       listOfExports=['schematise','genomeToPangenome','annotationToGenome'],
                       zoomLevels=[1],maxLengthComponent=100,maxLengthChunk=20,invertionThreshold=0.5,
                       debug=False,returnDebugData=False):
    
    if graph is None:
        if inputPath is not None:
            print('Loading Genome')
            graph = GenomeGraph(gfaPath=inputPath,isGFASeq=isSeq,**GenomeGraphParams)
        else:
            raise ValueError("Either graph or inputpath to GFA file should be provided")
    
    if outputPath is None and outputName is None:
        if inputPath is not None:
            if outputSuffix is not None:
                outputPath,outputName = pathConvert(inputPath,suffix=outputSuffix)
            else:
                outputPath,outputName = pathConvert(inputPath)
                
            
        else:
            raise ValueError("If inputPath is not given, then outputPath and outputName should be provided.")
    else:
        if outputSuffix is not None:
            outputName = outputName + outputSuffix
    print(f'Recording Pantograph data to {outputPath}{os.path.sep}{outputName}')

    numNodes = len(graph.nodes)
    numNodesDigits = np.int(np.ceil(np.log10(numNodes)))
    fromLinks = {}
    toLinks = {}

    startTime = time.time()

    rootStruct = deepcopy(rootStructTemplate)
    rootStruct["pathNames"] = graph.accessions
    
    if returnDebugData:
        zoomComponentLengths = {}
        zoomNodeToComponent = {}

        zoomComponentToNodes = {}
        zoomComponents = {} #temporary structure for testing, later each zoomlevel should be just saved as soon as it is processed.
        zoomCompNucleotides = {}
    
    nodeLengths = calcNodeLengths(graph)
    
    pathLengths,pathNodeArray,pathNodeLengths,pathDirArray,pathNodeLengthsCum = initialPathAnalysis(graph,nodeLengths)
    
    for zoomLevel in zoomLevels:
        
        zoomTime = time.time()
        
        os.makedirs(f'{outputPath}{os.path.sep}{outputName}{os.path.sep}{zoomLevel}',exist_ok=True)

        print('\n===========================')
        print(f'Zoom level {zoomLevel}')
        print('===========================')
        zoom_level_struct = {}
        zoom_level_struct["files"] = []
        
        
        
        if returnDebugData:
            nodeToComponent = zoomNodeToComponent.setdefault(zoomLevel,[])
            componentToNode = zoomComponentToNodes.setdefault(zoomLevel,[])
            componentLengths = zoomComponentLengths.setdefault(zoomLevel,[])
            components = zoomComponents.setdefault(zoomLevel,[])
            componentNucleotides = zoomCompNucleotides.setdefault(zoomLevel,[])
        else:
            nodeToComponent = [] #zoomNodeToComponent.setdefault(zoomLevel,[])
            componentToNode = [] #zoomComponentToNodes.setdefault(zoomLevel,[])
            componentLengths = [] #zoomComponentLengths.setdefault(zoomLevel,[])
            components = [] #zoomComponents.setdefault(zoomLevel,[])
            componentNucleotides = [] #zoomCompNucleotides.setdefault(zoomLevel,[])
        
        component = deepcopy(componentTemplate)

        occupants = set()
        nucleotides = ''
        matrix = {}
        
        
        nodeLinks = []
        
        nBins = 0
        previousInv = {}
        nCols = 0
        binLength = 0
        occupancy = {}
        inversion = {}
        pos = {}
        nodesInComp = set()
        breakComponentWhenBinEnds = False
        breakCompBeforeBin = False
        binOpen = False
        breakComponent = False
        forceBreak = False

        for nodeIdx in range(1,numNodes+1):
            if debug:    
                print(f'Processing node {nodeIdx:0{numNodesDigits}}/{numNodes:0{numNodesDigits}}')
            else:
                print(f'\rProcessing node {nodeIdx:0{numNodesDigits}}/{numNodes:0{numNodesDigits}}',end='')

            nodePathsIdx,nodeSeqInPath = np.where(pathNodeArray==nodeIdx)
            nodeLen = nodeLengths[nodeIdx-1]

            uniqueNodePathsIDx,pathNodeCount = np.unique(nodePathsIdx,return_counts=True)
            
            incomingFarLink = False
            pathStart = False
            pathEnd = False
            breakCompBeforeBin = False
            for j,pathID in enumerate(uniqueNodePathsIDx):
                occupants.add(pathID)

                pOcc = occupancy.setdefault(pathID,0) 
                pOcc += pathNodeCount[j]*nodeLengths[nodeIdx-1]
                occupancy[pathID] = pOcc # Occupancy
                
                localInv = 0
                for nodeNumInPath in nodeSeqInPath[np.where(nodePathsIdx==pathID)[0]]:
                    localInv += int(pathDirArray[pathID,nodeNumInPath])

                pInv = inversion.get(pathID,0) 
                pInv += localInv*nodeLengths[nodeIdx-1]/pathNodeCount[j]
                inversion[pathID] = pInv
                if (pInv-invertionThreshold)*(previousInv.setdefault(pathID,pInv)-invertionThreshold)<0:
                    breakCompBeforeBin = True
                previousInv[pathID] = pInv

                nodePositions = np.where(nodePathsIdx==pathID)[0]
                nodeInversionInPath = np.sum([pathDirArray[pathID,nodeSeqInPath[nodePos]] for nodePos in nodePositions])/len(nodePositions)
                for nodePos in nodePositions:
                    nodePositionInPath = nodeSeqInPath[nodePos]
                    if debug:
                        print(f'Node {nodeIdx}, Path {pathID} inversion {nodeInversionInPath}')
                    if nodeInversionInPath<=invertionThreshold:

                        if nodePositionInPath>0 and nodePositionInPath<pathLengths[pathID]-1:
                            if debug:
                                print(f"IncomingLinks: {pathNodeArray[pathID,nodePositionInPath-1]} -> {pathNodeArray[pathID,nodePositionInPath]}")
                                print(f'Existing known links: {fromLinks.get(pathNodeArray[pathID,nodePositionInPath-1],{})}')

                            previousNode = pathNodeArray[pathID,nodePositionInPath-1]
                            nextNode = pathNodeArray[pathID,nodePositionInPath]
                            if previousNode+1!=nextNode:
                                if (previousNode<nextNode and nextNode in fromLinks.get(previousNode,{})) or (previousNode>nextNode):
                                    incomingFarLink = True
                                    break
                        elif nodePositionInPath==0:
                            pathStart = True
                    else:

                        if nodePositionInPath>0 and nodePositionInPath<pathLengths[pathID]-1:
                            if debug:
                                print(f"IncomingLinks: {pathNodeArray[pathID,nodePositionInPath]} -> {pathNodeArray[pathID,nodePositionInPath+1]}")
                                print(f'Existing known links: {toLinks.get(pathNodeArray[pathID,nodePositionInPath+1],{})}')

                            previousNode = pathNodeArray[pathID,nodePositionInPath]
                            nextNode = pathNodeArray[pathID,nodePositionInPath+1]
                            if previousNode-1!=nextNode:
                                if (previousNode>nextNode and nextNode in toLinks.get(nextNode,{})) or (previousNode<nextNode):
                                    incomingFarLink = True
                                    break
                        elif nodePositionInPath==pathLengths[pathID]-1:
                            pathEnd = True
#                 if incomingFarLink or pathStart or pathEnd:
#                     break

            if nodeIdx>1 and (pathStart or pathEnd or incomingFarLink or breakCompBeforeBin) and len(matrix)>0:
                if debug:
                    if incomingFarLink:
                        print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to incoming far link.')
                    else:
                        print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to start or end of a path.')
                component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = \
                    finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides)
                componentToNode.append(list(nodesInComp))
                nodesInComp = set([nodeIdx])
                breakComponent = False
                forceBreak = False
                breakCompBeforeBin = False
            
            nodeToComponent.append([len(components)])
            nodesInComp.add(nodeIdx)

            if zoomLevel<nodeLen:
                for posInd,i in enumerate(range(0,nodeLen,zoomLevel)):
                    
#                     breakCompBeforeBin = False
#                     for j,pathID in enumerate(uniqueNodePathsIDx):
#                         occupants.add(pathID)

#                         pOcc = occupancy.setdefault(pathID,0) 
#                         pOcc += pathNodeCount[j]*binLength
#                         occupancy[pathID] = pOcc # Occupancy

#                         posPath = pos.setdefault(pathID,[])
#                         localInv = 0
#                         for nodeNumInPath in nodeSeqInPath[np.where(nodePathsIdx==pathID)[0]]:
#                             localInv += int(pathDirArray[pathID,nodeNumInPath])
#                             offset = np.min([i+zoomLevel,nodeLen])
#                             nodeStart = pathNodeLengthsCum[pathID,nodeNumInPath]-nodeLen+1
#                             posPath.append([nodeStart+i,nodeStart+offset-1])


#                         pInv = inversion.get(pathID,0) 
#                         pInv += localInv*binLength/pathNodeCount[j]
#                         inversion[pathID] = pInv
#                         if (pInv-invertionThreshold)*(previousInv.setdefault(pathID,pInv)-invertionThreshold)<0:
#                             breakCompBeforeBin = True
#                         previousInv[pathID] = pInv

#                     if breakCompBeforeBin and len(matrix)>0:
#                         nodeToComponent[-1].remove(len(components))
#                         nodesInComp.remove(nodeIdx)

#                         component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = \
#                         finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides)                    

#                         nodeToComponent[-1].append(len(components))
#                         componentToNode.append(list(nodesInComp))
#                         nodesInComp = set([nodeIdx])
#                         breakCompBeforeBin = False
#                         breakComponent = False
#                         forceBreak = False
                    
                    if zoomLevel==1:
                        if isSeq:
                            nucleotides += graph.nodesData[nodeIdx-1][i:i+zoomLevel]
                        else:
                            nucleotides += graph.nodes[nodeIdx-1][i:i+zoomLevel]

                    if (i+zoomLevel>nodeLen):
                        binLength = nodeLen-i
                    else:
                        binLength = zoomLevel

                    for j,pathID in enumerate(uniqueNodePathsIDx):
                        posPath = pos.setdefault(pathID,[])
                        for nodeNumInPath in nodeSeqInPath[np.where(nodePathsIdx==pathID)[0]]:
                            offset = np.min([i+zoomLevel,nodeLen])
                            nodeStart = pathNodeLengthsCum[pathID,nodeNumInPath]-nodeLen+1
                            posPath.append([nodeStart+i,nodeStart+offset-1])

                    matrix,pos,binLength,_,_,nBins,nCols,binOpen = \
                        finaliseBin(matrix,pos,binLength,nodeLengths[nodeIdx-1],occupancy,inversion,nBins,nCols,posInd)

                    if nBins+1>maxLengthComponent and (i+zoomLevel<nodeLen) and len(matrix)>0:
                        # Because this is intra-node break, all connectors will be handled by rdepartures without separating into inverted or non-inverted. 
                        # All inter-node links will be handled through links later when chunks are being recorded.
                        
                        forwardPaths = []
                        invertedPaths = []
                        for pathID in range(len(graph.paths)):
                            if pathID in occupants:
                                if matrix[pathID][1][0][1]>invertionThreshold:
                                    invertedPaths.append(pathID)
                                else:
                                    forwardPaths.append(pathID)
                            else:
                                forwardPaths.append(pathID)
                        
                        if len(forwardPaths)>0:
                            
                            component["rdepartures"].append({
                                "upstream": component["first_bin"]+nBins-1,
                                "downstream": component["first_bin"]+nBins,
                                "participants": forwardPaths,
                                'otherSideRight': False
                            })
                        if len(invertedPaths)>0:
                            component["rarrivals"].append({
                                "upstream": component["first_bin"]+nBins,
                                "downstream": component["first_bin"]+nBins-1,
                                "participants": invertedPaths,
                                'otherSideRight': False
                            })

                        if debug:
                            print(f'Node {nodeIdx}: Component broken inside node {nodeIdx} due to max component length.')

                        component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides)                    

                        nodeToComponent[-1].append(len(components))
                        componentToNode.append(list(nodesInComp))
                        nodesInComp = set([nodeIdx])
                        breakComponent = False
                        forceBreak = False
                binLength = 0
                occupancy = {}
                inversion = {}
            else:
                if nBins==0 and not binOpen:
                    if debug:
                        print(f'Before processing node {nodeIdx} the nodeLinks was cleared')
                    nodeLinks = []
                binOpen = True
                binLength += nodeLen

#                 if isSeq:
#                     nucleotides += graph.nodesData[nodeIdx-1]
#                 else:
#                     nucleotides += graph.nodes[nodeIdx-1]

#                 breakCompBeforeBin = False
#                 for j,pathID in enumerate(uniqueNodePathsIDx):

#                     occupants.add(pathID)

#                     pOcc = occupancy.setdefault(pathID,0) 
#                     pOcc += pathNodeCount[j]*nodeLen
#                     occupancy[pathID] = pOcc

#                     localInv = 0
#                     posPath = pos.setdefault(pathID,[])
#                     for nodeNumInPath in nodeSeqInPath[np.where(nodePathsIdx==pathID)[0]]:
#                         localInv += int(pathDirArray[pathID,nodeNumInPath])
#                         nodeStart = pathNodeLengthsCum[pathID,nodeNumInPath]-nodeLen+1
#                         posPath.append([nodeStart,nodeStart+nodeLen-1])
#                     pInv = inversion.get(pathID,0) 
#                     pInv += localInv*nodeLen/pathNodeCount[j]
#                     inversion[pathID] = pInv
#                     if (pInv-invertionThreshold)*(previousInv.setdefault(pathID,0)-invertionThreshold)<0:
#                         breakCompBeforeBin = True
#                     previousInv[pathID] = pInv

#                 if breakCompBeforeBin and len(matrix)>0:
#                     nodeToComponent[-1].remove(len(components))
#                     nodesInComp.remove(nodeIdx)

#                     component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = \
#                     finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides)                    

#                     nodeToComponent[-1].append(len(components))
#                     componentToNode.append(list(nodesInComp))
#                     nodesInComp = set([nodeIdx])
#                     breakCompBeforeBin = False
#                     breakComponent = False
#                     forceBreak = False
                
                for j,pathID in enumerate(uniqueNodePathsIDx):
                    posPath = pos.setdefault(pathID,[])
                    for nodeNumInPath in nodeSeqInPath[np.where(nodePathsIdx==pathID)[0]]:
                        nodeStart = pathNodeLengthsCum[pathID,nodeNumInPath]-nodeLen+1
                        posPath.append([nodeStart,nodeStart+nodeLen-1])

                if nodeIdx<len(graph.nodes):
                    if binLength+nodeLengths[nodeIdx]>zoomLevel:
                        matrix,pos,binLength,occupancy,inversion,nBins,nCols,binOpen = finaliseBin(matrix,pos,binLength,binLength,occupancy,inversion,nBins,nCols,0)
                else:
                    matrix,pos,binLength,occupancy,inversion,nBins,nCols,binOpen = finaliseBin(matrix,pos,binLength,binLength,occupancy,inversion,nBins,nCols,0)

            # marking path ends and breaking component if any exist
            pathEnds = np.sort(np.unique([pathIdx for pathIdx,nodeInSeq in zip(nodePathsIdx,nodeSeqInPath) if pathLengths[pathIdx]==nodeInSeq+1]))
            if len(pathEnds)>0:
                if binOpen:
                    component.setdefault("ends",set()).update(pathEnds.tolist())
                else:
                    if nBins>0:

                        component.setdefault("ends",set()).update(pathEnds.tolist())

                    else:
                        components[-1].setdefault("ends",set()).update(pathEnds.tolist())

            pathEndsRelevant = np.sort(np.unique([pathIdx for pathIdx,nodeInSeq in zip(nodePathsIdx,nodeSeqInPath) if pathLengths[pathIdx]==nodeInSeq+1 and (not pathDirArray[pathIdx,nodeInSeq])]))
            pathStartsRelevant = np.sort(np.unique([pathIdx for pathIdx,nodeInSeq in zip(nodePathsIdx,nodeSeqInPath) if nodeInSeq==0 and pathDirArray[pathIdx,nodeInSeq]]))
            if len(pathEndsRelevant)>0 or len(pathStartsRelevant)>0:
                if binOpen:
                    if debug:
                        print(f'Node {nodeIdx}: Component will be broken due to start or end of a path at node {nodeIdx} when bin is filled.')
                    breakComponent = True
                    forceBreak = True
                elif nBins>0:
                    if debug:
                        print(f'Node {nodeIdx}: Component broken after node {nodeIdx} due to start or end of a path.')
                    component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides)
                    componentToNode.append(list(nodesInComp))

                    nodesInComp = set()
                    breakComponent = False
                    forceBreak = False

            if nodeIdx<len(graph.nodes) and nBins>0:
                if nBins+nodeLengths[nodeIdx]/zoomLevel>maxLengthComponent:

                    if debug:
                        print(f'Node {nodeIdx}: Component broken after node {nodeIdx} because next component will not fit in this component.')
                    component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides)
                    componentToNode.append(list(nodesInComp))

                    nodesInComp = set()
                    breakComponent = False
                    forceBreak = False
            elif nBins>0:

                if debug:
                    print(f'Node {nodeIdx}: Last node in the last component.')
                component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides)
                componentToNode.append(list(nodesInComp))

                nodesInComp = set()
                breakComponent = False


            nodeFromLink = fromLinks.setdefault(nodeIdx,{})

            if zoomLevel==1:
                for j,pathID in enumerate(uniqueNodePathsIDx):
                    nodePositions = np.where(nodePathsIdx==pathID)[0]
                    nodeInversionInPath = np.sum([pathDirArray[pathID,nodeSeqInPath[nodePos]] for nodePos in nodePositions])/len(nodePositions)
                    for nodePos in nodePositions:
                        nodePositionInPath = nodeSeqInPath[nodePos]
                        if nodePositionInPath<(pathLengths[pathID]-1):
                            nextNode = pathNodeArray[pathID,nodePositionInPath+1]
                            nextNodePathsIdx,nextNodeSeqInPath = np.where(pathNodeArray==nextNode)
                            nextNodePositions = np.where(nextNodePathsIdx==pathID)[0]
                            nextNodeInversionInPath = np.sum([pathDirArray[pathID,nextNodeSeqInPath[nodePos]] for nodePos in nextNodePositions])/len(nextNodePositions)
                            if debug:
                                print(f'Node {nodeIdx}, next node {nextNode}, path {pathID}, number in path {nodePositionInPath}, length of path {pathLengths[pathID]}')

                            if nodeInversionInPath>invertionThreshold and nextNodeInversionInPath>invertionThreshold:
                                reverseCond  = (nodeIdx<=nextNode)
                                step = -1
                            else:
                                reverseCond = (nodeIdx>=nextNode)
                                step = 1

                            if reverseCond:
                                if debug:
                                    print(f'Reverse link for path {pathID} from node {nodeIdx} to node {nextNode}')

                                nodeFromLink.setdefault(nextNode,[]).append(pathID)
                                toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
                                if (nBins>0 or binOpen) and (nodeInversionInPath<=0.5):

                                    if debug:
                                        print(f'Node {nodeIdx}: Component will be broken after node {nodeIdx} due to backward link to node {nextNode}.')
                                    nodeLinks.append(nextNode)
                                    breakComponent = True
                            else:
                                if np.any([node in pathNodeArray[pathID,:] for node in range(nodeIdx+1*step,nextNode,step)]):
                                    if debug:
                                        print(f'Jump link for path {pathID} from node {nodeIdx} to node {nextNode}')

                                    nodeFromLink.setdefault(nextNode,[]).append(pathID)
                                    toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
                                    if (nBins>0 or binOpen) and (nodeInversionInPath<=0.5): 

                                        if debug:
                                            print(f'Node {nodeIdx}: Component will be broken after node {nodeIdx} due forward jumping link to node {nextNode}.')
                                        nodeLinks.append(nextNode)
                                        breakComponent = True
                                else:


                                    startNode = None
                                    endNode = None
                                    step = None

                                    if nodeInversionInPath>invertionThreshold and nextNodeInversionInPath>invertionThreshold:
                                        startNode = nodeIdx
                                        endNode = nextNode
                                        step = -1
                                        if debug:
                                            print(f'Intermediate nodes for path {pathID} from node {nodeIdx} to node {nextNode}')

                                    elif nodeInversionInPath<=invertionThreshold and nextNodeInversionInPath<=invertionThreshold:
                                        startNode = nodeIdx
                                        endNode = nextNode
                                        step = 1
                                        if debug:
                                            print(f'Intermediate nodes for path {pathID} from node {nodeIdx} to node {nextNode}')
                                    else:
                                        if debug:
                                            print(f'Jump link for path {pathID} from node {nodeIdx} to node {nextNode}')

                                        nodeFromLink.setdefault(nextNode,[]).append(pathID)
                                        toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
                                        if nBins>0 or binOpen: 

                                            if debug:
                                                print(f'Node {nodeIdx}: Component will be broken after node {nodeIdx} due forward jumping link to node {nextNode}.')
                                            nodeLinks.append(nextNode)
                                            breakComponent = True

                                    if startNode is not None and endNode is not None and step is not None:
                                        for intermediateNodeIdx in range(startNode,endNode,step):
                                            if debug:
                                                print(f'Adding link from node {intermediateNodeIdx} to node {intermediateNodeIdx+1*step} for path {pathID}')
                                            fromLinks.setdefault(intermediateNodeIdx,{}).setdefault(intermediateNodeIdx+1*step,[]).append(pathID)
                                            toLinks.setdefault(intermediateNodeIdx+1*step,{}).setdefault(intermediateNodeIdx,[]).append(pathID)
                        else:
                            if debug:
                                print("Last node in path")
            else:#if not breakComponent:
                if nBins>0 or binOpen:
                    breakLink,nnodes = checkLinksForBreak(nodeIdx,nodeFromLink,toLinks.setdefault(nodeIdx,{}),matrix,inversion,binOpen,invertionThreshold,True)
                    if breakLink==2 or (not binOpen and breakLink==1):
                        if debug:
                            print(f'Node {nodeIdx}: Component will be broken after node {nodeIdx} due a link from that node (res code {breakLink}) to nodes {nnodes}.')
                        breakComponent = True
                        nodeLinks.extend(nnodes)

            if breakComponent and not binOpen:
                if np.any([node not in nodesInComp for node in nodeLinks]) or len(nodeLinks)==0 or forceBreak:
                    
                    if debug:
                        print(f'Node {nodeIdx}: Component broken after node {nodeIdx} because of previously set flag.')
                        print(f'Nodes in component: {nodesInComp}')
                        print(f'Nodes linked from component: {nodeLinks}')
                    component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides)
                    componentToNode.append(list(nodesInComp))
                    nodesInComp = set()
                    breakComponent = False
                    forceBreak = False
                    nodeLinks = []
                else:
                    if debug:
                        print(f'Node {nodeIdx}: Component is not broken as all links are internal, flag reset.')
                        print(f'Nodes in component: {nodesInComp}')
                        print(f'Nodes linked from component: {nodeLinks}')
                    breakComponent = False;

        print('\nNodes processed, recording component to files.')

        if zoomLevel==1:
            rootStruct["pangenome_length"] = np.sum(componentLengths)

        chunkList = rootStruct["zoom_levels"].setdefault(zoomLevel,{
            "last_bin": components[-1]["last_bin"],
            "files":[]
        })
        # Recording data to files for zoom level

    #     chunkTemplate = {
    #     "json_version":17,
    #     "bin_width":1,
    #     "first_bin":1,
    #     "includes_connectors": True,
    #     "components": []
    # }    
        chunk = deepcopy(chunkTemplate)
        chunkNum = 0
        prevTotalCols = 0
        curCompCols = 0
        nucleotides = ''
        nBins = 0
        numComps = len(components)
        numCompsDigits = np.int(np.ceil(np.log10(numComps)))
        
        for compNum in range(numComps):
            if debug:    
                print(f'Processing component {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}')
            else:
                print(f'\rProcessing component {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}',end='')
                
            component = components[compNum]

            nodeInComp = componentToNode[compNum]
            nodeInComp.sort()

            if len(nodeInComp)>1:
                component = addLinksToComp(nodeInComp,nodeToComponent,toLinks,fromLinks,component,components)
            elif len(nodeInComp)==1:

                mainNode = nodeInComp[0]
    #             compForNode = nodeToComponent[mainNode-1]
                doLeft = False
                if compNum==0:
                    doLeft = True
                elif componentToNode[compNum-1][-1]!=mainNode:
                    doLeft = True

                doRight = False # Check this conditions as they may be causing issue!
                if compNum==(len(components)-1):
                    doRight = True
                elif componentToNode[compNum+1][0]!=mainNode:
                    doRight = True

                component = addLinksToComp(nodeInComp,nodeToComponent,toLinks,fromLinks,component,components,doLeft,doRight)

            else:
                raise ValueError(f"Component {compNum} does not have any associated nodes!")
            
            nBins += component["last_bin"]-component["first_bin"]+1
            if len(chunk['components'])>0:
                if checkLinks(chunk["components"][-1],component):
                    
                    newComp = joinComponents(chunk["components"].pop(),component,maxLengthComponent,invertionThreshold)
                    if isinstance(newComp,list):
                        chunk["components"].append(newComp[0])
                        component = newComp[1]
                    else:
                        component = newComp
            
            nucleotides += componentNucleotides[compNum]
            chunk["components"].append(component)
            

            # End of chunk
            if compNum<len(components)-1:

                if len(chunk['components'])>=maxLengthChunk:
                #nBins+components[compNum+1]["last_bin"]-components[compNum+1]["first_bin"]+1>maxLengthChunk:
                    rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols,nucleotides = finaliseChunk(
                                                                                              rootStruct,
                                                                                              zoomLevel,
                                                                                              chunk,
                                                                                              nucleotides,
                                                                                              nBins,
                                                                                              chunkNum,
                                                                                              curCompCols,
                                                                                              prevTotalCols,
                                                                                              outputPath,
                                                                                              outputName)
            else:
                rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols,nucleotides = finaliseChunk(
                                                                                          rootStruct,
                                                                                          zoomLevel,
                                                                                          chunk,
                                                                                          nucleotides,
                                                                                          nBins,
                                                                                          chunkNum,
                                                                                          curCompCols,
                                                                                          prevTotalCols,
                                                                                          outputPath,
                                                                                          outputName)
        if not debug:
            print()
        print(f'Recording finished. Zoom level time is {time.time() - zoomTime}. Time elapsed {time.time() - startTime}')

    with open(f'{outputPath}{os.path.sep}{outputName}{os.path.sep}bin2file.json','w') as f:
        json.dump(rootStruct,f,cls=NpEncoder)
    
    if returnDebugData:
        return zoomComponentLengths,zoomNodeToComponent,zoomComponentToNodes,zoomComponents,zoomCompNucleotides

In [ ]:
#export
def checkLinks(leftComp,rightComp):
    leftRdepCond = np.all([link['upstream']+1==link['downstream'] for link in leftComp['rdepartures']])
    leftRarrCond = np.all([link['upstream']-1==link['downstream'] for link in leftComp['rarrivals']])

    rightRdepCond = np.all([link['upstream']-1==link['downstream'] for link in rightComp['ldepartures']])
    rightRarrCond = np.all([link['upstream']+1==link['downstream'] for link in rightComp['larrivals']])
    
    return leftRdepCond and leftRarrCond and rightRarrCond and rightRdepCond

In [ ]:
#export
def getMatrixPathElement(matrix,pathID):
    res = [el for el in matrix if el[0]==pathID]
    if len(res)==1:
        return res[0]
    elif len(res)>0:
        warnings.warn(f"More than one element for path {pathID} is found!")
    
    return None
        

In [ ]:
#export
def joinComponents(leftComp,rightComp, maxLengthComponent, invertionThreshold=0.5):
    '''
    If the joining was successful, the function will return a joined component.
    
    If the joining was not successful and was aborted for one of the following reasons, it will return a list of original components. 
    The reasons for aborting the joining can be the following:
    - In one of the paths the invertion is lower than threshold in one component and higher in the other.
    - Left component contains at least one end
    - Right component contains at least one start
    
    The function will not check links for coming or going on the right of the left component and left of the right component. 
    It will just get left links from left component and right links from right component and assign them to the new component.
    '''
    
    if leftComp['last_bin']-leftComp['first_bin']+1 + rightComp['last_bin']-rightComp['first_bin']+1 > maxLengthComponent:
        return [leftComp,rightComp]
    
    if leftComp.get('ends',False):
        # End of a path
        return [leftComp,rightComp]
    
    newComp = {}
    newComp['first_bin'] = min(leftComp['first_bin'],rightComp['first_bin'])
    newComp['last_bin'] = max(leftComp['last_bin'],rightComp['last_bin'])
    newComp['firstCol'] = min(leftComp['firstCol'],rightComp['firstCol'])
    newComp['lastCol'] = max(leftComp['lastCol'],rightComp['lastCol'])
    
    leftCompNumBins = leftComp['last_bin']-leftComp['first_bin']+1
    
    newComp['occupants'] = list(set(leftComp['occupants']).union(rightComp['occupants']))
    
    for pathID in newComp['occupants']:
        leftPathElement = getMatrixPathElement(leftComp['matrix'],pathID)
        rightPathElement = getMatrixPathElement(rightComp['matrix'],pathID)
        if leftPathElement is None:
            if len([el for el in rightPathElement[2][1] if el[2][0][0]==1 or el[2][-1][0]==1])>0:
                # Start of a path
                return [leftComp,rightComp]
            newComp.setdefault("matrix",[]).append(rightPathElement)
            continue
        
        if rightPathElement is None:
            newComp.setdefault("matrix",[]).append(leftPathElement)
            continue
        
        if (leftPathElement[1]>invertionThreshold and rightPathElement[1]<=invertionThreshold) or \
           (leftPathElement[1]<=invertionThreshold and rightPathElement[1]>invertionThreshold):
            return [leftComp,rightComp]
        
        newPathElement = []
        newPathElement.append(pathID)
        newPathElement.append(leftPathElement[1])
        pathMatrix = []
        pathMatrix.append(leftPathElement[2][0] + [el+leftCompNumBins for el in rightPathElement[2][0]])
        pathMatrix.append(leftPathElement[2][1] + rightPathElement[2][1])
        newPathElement.append(pathMatrix)
        newComp.setdefault("matrix",[]).append(newPathElement)
        
    newComp['larrivals'] = leftComp['larrivals']
    newComp['ldepartures'] = leftComp['ldepartures']
    newComp['rarrivals'] = rightComp['rarrivals']
    newComp['rdepartures'] = rightComp['rdepartures']
    
    return newComp

In [ ]:
#export
def calcNodeLengths(graph):
    print('Calculating nodes length...')
    
    numNodes = len(graph.nodes)
    numNodesDigits = np.int(np.ceil(np.log10(numNodes)))
    nodeLengths = [0]*numNodes
    
    for nodeIdx in range(numNodes):
        print(f'\rProcessing node {nodeIdx+1:0{numNodesDigits}}/{numNodes:0{numNodesDigits}}',end='')
        if graph.nodesData[nodeIdx]=='':
            nodeLengths[nodeIdx] = len(graph.nodes[nodeIdx])
        else:
            nodeLengths[nodeIdx] = len(graph.nodesData[nodeIdx])
    
    print('\nFinished calculating nodes lengths')
    return nodeLengths

In [ ]:
#export
def initialPathAnalysis(graph,nodeLengths):
    print('Preprocessing paths...')
    
    numPaths = len(graph.paths)
    numPathsZeros = np.int(np.ceil(np.log10(numPaths)))
    
    maxLengthPath = len(max(graph.paths,key=lambda arr: len(arr)))

    pathLengths = []
    pathNodeArray = np.zeros((len(graph.paths),maxLengthPath),dtype = np.int)
    pathNodeLengths = np.zeros((len(graph.paths),maxLengthPath),dtype = np.int)
    pathDirArray = np.zeros((len(graph.paths),maxLengthPath),dtype = np.bool) # True - "+", False - "-" or padding for shorter paths.

    for i,path in enumerate(graph.paths):
        print(f'\rProcessing path {i+1:0{numPathsZeros}}/{numPaths:0{numPathsZeros}}',end='')
        pathLengths.append(len(path))

        pathNodeArray[i,:pathLengths[-1]] = [int(node[:-1]) for node in path]
        pathDirArray[i,:pathLengths[-1]] = [node[-1]=='-' for node in path]
        pathNodeLengths[i,:pathLengths[-1]] = [nodeLengths[node-1] for node in pathNodeArray[i,:pathLengths[-1]]]

    pathNodeLengthsCum = np.cumsum(pathNodeLengths,axis=1)
    print('\nFinished preprocessing paths')
    return pathLengths,pathNodeArray,pathNodeLengths,pathDirArray,pathNodeLengthsCum

In [ ]:
#export
def finaliseBin(matrix,pos,binLength,occInvAdj,occupancy,inversion,nBins,nCols,posInd):
    for pathID in pos.keys():
        matrixPath = matrix.setdefault(pathID,[[],[]])
        posPath = pos.get(pathID)
        posArray = np.array(posPath)
#         print(f'[finaliseBin] original pos {posArray}')
        posArray = posArray[np.argsort(posArray[:,0]),:]
        posIntersect = (posArray[1:,1]-(posArray[:-1,0]-1))*\
                        (posArray[:-1,1]-(posArray[1:,0]-1))
#         print(f'[finaliseBin] pos intersections {posIntersect}')
        newPos = [[posArray[0,0]]]
        candidates = [posArray[0,1]]
        for jointNum in range(len(posIntersect)):
            if posIntersect[jointNum]>=0:
                candidates.extend(posArray[jointNum+1,:].tolist())
            else:
                newPos[-1].append(np.max(candidates))
                newPos.append([posArray[jointNum+1,0]])
                candidates = [posArray[jointNum+1,1]]
            
        newPos[-1].append(np.max(candidates))# !!!!        
        
#         print(f'[finaliseBin] joined pos {newPos}')
        
        inversionRate = inversion[pathID]/occInvAdj
    
        matrixPathRecord = [0,0,[]]
        matrixPath[0].append(nBins)
        matrixPathRecord[0] = occupancy[pathID]/occInvAdj # Occupancy
        matrixPathRecord[1] = inversionRate # Inversion rate
        matrixPathRecord[2] = newPos
        if inversionRate>0.5:
            matrixPath[1].insert(posInd,matrixPathRecord)
        else:
            matrixPath[1].append(matrixPathRecord)
        
    return matrix,{},0,{},{},nBins+1,nCols+binLength,False #matrix,pos,binLength,occupancy,inversion,nBins,nCols,binOpen

In [ ]:
#export
def adjustMatrixPathArray(matrixPathArray,isInverted,nBins):
    if isInverted:
        _tempArray = [nBins-binNum-1 for binNum in matrixPathArray[0]]
        _tempArray.sort()
        matrixPathArray[0] = _tempArray
    return matrixPathArray

In [ ]:
#export
def finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides,invertionThreshold=0.5):
    componentLengths.append(nBins)
    component["matrix"].extend([[pathID,
                                 int(matrixPathArray[1][0][1]>invertionThreshold),
                                 adjustMatrixPathArray(matrixPathArray,int(matrixPathArray[1][0][1]>invertionThreshold),nBins)] \
                                for pathID,matrixPathArray in matrix.items()])
    component["occupants"] = sorted(list(occupants))
    component['last_bin'] = component['first_bin'] + nBins - 1
    component['lastCol'] = component['firstCol'] + nCols - 1
    componentNucleotides.append(nucleotides)
    firstBin = component['last_bin'] + 1
    firstCol = component['lastCol'] + 1
    components.append(component)
    component = deepcopy(componentTemplate)
    component['first_bin'] = firstBin
    component['firstCol'] = firstCol
    return component,components,componentNucleotides,{},set(),0,0,''

In [ ]:
#export
def checkLinksForBreak(nodeIdx,nodeFromLinks,nodeToLinks,matrix,inversion,binOpen,inversionThreshold,isCurrent):
    res = 0
    nodes = set()
    if isCurrent:
        for node,pathList in nodeFromLinks.items():
            if nodeIdx==node:
                res = max(1,res)
                nodes.add(node)
            else:
                for pathID in pathList:
                    if binOpen:
                        invLevel = inversion.get(pathID)
                    else:
                        invLevel = matrix.get(pathID)
                        if invLevel is not None:
                            invLevel = invLevel[1][0][1]
                    if invLevel is not None:
                        if invLevel<=inversionThreshold:
                            # Forward path
                            if nodeIdx+1!=node:
                                res = max(2,res)
                                nodes.add(node)
        for node,pathList in nodeToLinks.items():
            if nodeIdx==node:
                res = max(1,res)
                nodes.add(node)
            else:
                for pathID in pathList:
                    if binOpen:
                        invLevel = inversion.get(pathID)
                    else:
                        invLevel = matrix.get(pathID)
                        if invLevel is not None:
                            invLevel = invLevel[1][0][1]
                    if invLevel is not None:
                        if invLevel>inversionThreshold:
                            # Inverted path
                            if nodeIdx+1!=node:
                                res = max(2,res)
                                nodes.add(node)
            
    else:
        # Do not use isCurrent=False !!!
        for node,pathList in nodeFromLinks.items():
            if nodeIdx==node:
                res = max(1,res)
                nodes.add(node)
            else:
                for pathID in pathList:
                    if binOpen:
                        invLevel = inversion.get(pathID)
                    else:
                        invLevel = matrix.get(pathID)
                        if invLevel is not None:
                            invLevel = invLevel[1][0][1]
                    if invLevel is not None:
                        if invLevel>inversionThreshold:
                            # Inverted path
                            if nodeIdx-1!=node:
                                res = max(2,res)
                                nodes.add(node)
        for node,pathList in nodeToLinks.items():
            if nodeIdx==node:
                res = max(1,res)
                nodes.add(node)
            else:
                for pathID in pathList:
                    if binOpen:
                        invLevel = inversion.get(pathID)
                    else:
                        invLevel = matrix.get(pathID)
                        if invLevel is not None:
                            invLevel = invLevel[1][0][1]
                    if invLevel is not None:
                        if invLevel<=inversionThreshold:
                            # Inverted path
                            if nodeIdx+1!=node:
                                res = max(2,res)
                                nodes.add(node)
        
    return res,list(nodes)

In [ ]:
#export
def finaliseChunk(rootStruct,zoomLevel,chunk,nucleotides,nBins,chunkNum,curCompCols,prevTotalCols,outputPath,outputName):
    endChunkBin = chunk['components'][-1]['last_bin']
    chunk['last_bin'] = endChunkBin
    
    localPath = f'{outputPath}{os.path.sep}{outputName}{os.path.sep}{zoomLevel}{os.path.sep}'
    
    fileName = f'chunk{chunkNum}_zoom{zoomLevel}.schematic.json'
    
    with open(f'{localPath}{fileName}','w') as f:
        json.dump(chunk,f,cls=NpEncoder)
    
    rootStruct['zoom_levels'][zoomLevel]['files'].append({
        'file': fileName,
        'first_bin':chunk['first_bin'],
        'last_bin':chunk['last_bin'],
#         'x':prevTotalCols
    })
    
    if zoomLevel==1:
        fastaName = f'seq_chunk{chunkNum}_zoom{zoomLevel}.fa'
        rootStruct['zoom_levels'][zoomLevel]['files'][-1]['fasta'] = fastaName
        
        with open(f'{localPath}{fastaName}','w') as f:
            f.write(f'>first_bin:{chunk["first_bin"]} last_bin:{chunk["last_bin"]}\n')
            f.write(nucleotides)
        
    chunk = deepcopy(chunkTemplate)
    chunk['first_bin'] = endChunkBin + 1
    return rootStruct,chunk,0,chunkNum + 1,prevTotalCols+curCompCols,0,'' #rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols

In [ ]:
#export
def addLinksToComp(nodeInComp,nodeToComponent,toLinks,fromLinks,component,components,doLeft=True,doRight=True):
    compLinks = fillLinks(nodeInComp,nodeToComponent,toLinks,fromLinks,component,components,doLeft,doRight)
    if "lArrivals" in compLinks:
        for (upstream,downstream,otherSide),participants in compLinks["lArrivals"].items():
            if otherSide=='fr':
                fromRight = True
            else:
                fromRight = False
            component['larrivals'].append({
                    'upstream': upstream,
                    'downstream': downstream,
                    'otherSideRight': fromRight,
                    'participants': participants
                })
    
    if 'rArrivals' in compLinks:
        for (upstream,downstream,otherSide),participants in compLinks["rArrivals"].items():
            if otherSide == 'fr':
                fromRight = True
            else:
                fromRight = False
            component['rarrivals'].append({
                    'upstream': upstream,
                    'downstream': downstream,
                    'otherSideRight': fromRight,
                    'participants': participants
                })
            
    if 'rDepartures' in compLinks:
        for (upstream,downstream,otherSide),participants in compLinks["rDepartures"].items():
            if otherSide == 'tl':
                toRight = False
            else:
                toRight = True
            component['rdepartures'].append({
                'upstream': upstream,
                'downstream': downstream,
                'otherSideRight': toRight,
                'participants': participants
            })
    
    if 'lDepartures' in compLinks:
        for (upstream,downstream,otherSide),participants in compLinks["lDepartures"].items():
            if otherSide == 'tl':
                toRight = False
            else:
                toRight = True
            component['ldepartures'].append({
                'upstream': upstream,
                'downstream': downstream,
                'otherSideRight': toRight,
                'participants': participants
            })
        
    return component

In [ ]:
#export
def splitforwardInversedNodeComp(pathList,component,isInverse):
#     if component['firstCol']==460:
#         pdb.set_trace()
    
    forward = []
    inversed = []
    
    for pathID in pathList:
            try:
                if component["matrix"][component["occupants"].index(pathID)][1]>0:
                    inversed.append(pathID)
                else:
                    forward.append(pathID)
            except (IndexError,ValueError):
                # If it is artificial pass link.
                if isInverse:
                    inversed.append(pathID)
                else:
                    forward.append(pathID)
                
    return forward,inversed

In [ ]:
#export
def fillLinks(nodeInComp,nodeToComponent,toLinks,fromLinks,component,components,doLeft=True,doRight=True):
    compLinks = {}
    
    
    
#     if component['first_bin']==2 and len(nodeInComp)==2:
#         pdb.set_trace()
    
    for node in nodeInComp:
        
        # Processing all external arrival links
        nodeToLink = toLinks.get(node,{})
        for fromNode in nodeToLink.keys():
            intermediateCondition = (node<fromNode)
            
            la,ra = splitforwardInversedNodeComp(nodeToLink[fromNode],component,intermediateCondition)
            
            fromFirstComp = components[nodeToComponent[fromNode-1][0]]
            fromLastComp = components[nodeToComponent[fromNode-1][-1]]
            
            #left arrivals
            if len(la)>0 and doLeft:
                frd,fld = splitforwardInversedNodeComp(la,fromFirstComp,intermediateCondition)
                
                complArrivals = compLinks.setdefault('lArrivals',{})
                # from right departure
                if len(frd)>0 and (fromNode not in nodeInComp) and (fromLastComp['last_bin']+1!=component['first_bin']):
                    complArrivals.setdefault((fromLastComp['last_bin'],component['first_bin'],'fr'),set()).update(frd)
                #from left departure
                if len(fld)>0 and (fromNode not in nodeInComp):
                    complArrivals.setdefault((fromFirstComp['first_bin'],component['first_bin'],'fl'),set()).update(fld)
            
            #right arrivals
            if len(ra)>0 and doRight:
                frd,fld = splitforwardInversedNodeComp(ra,fromFirstComp,intermediateCondition)
                
                comprArrivals = compLinks.setdefault('rArrivals',{})
                #from right departures
                if len(frd)>0 and (fromNode not in nodeInComp):
                    comprArrivals.setdefault((fromLastComp['last_bin'],component['last_bin'],'fr'),set()).update(frd)
                #from left departures
                if len(fld)>0 and (fromNode not in nodeInComp):
                    comprArrivals.setdefault((fromFirstComp['first_bin'],component['last_bin'],'fl'),set()).update(fld)
            
        # Processing all external departure links
        nodeFromLink = fromLinks.get(node,{})
        for toNode in nodeFromLink.keys():
            intermediateCondition = (node>toNode)
            
            rd,ld = splitforwardInversedNodeComp(nodeFromLink[toNode],component,intermediateCondition)
            
            toFirstComp = components[nodeToComponent[toNode-1][0]]
            toLastComp = components[nodeToComponent[toNode-1][-1]]
            
            #right departures
            if len(rd)>0 and doRight: # Check if doRight is set incorrectly for our case (121->122 at level 4)
                tla,tra = splitforwardInversedNodeComp(rd,toFirstComp,intermediateCondition)
                
                comprDepartures = compLinks.setdefault('rDepartures',{})
                #to left arrivals
                if len(tla)>0:
                    if toNode not in nodeInComp:
                         comprDepartures.setdefault((component['last_bin'],toFirstComp['first_bin'],'tl'),set()).update(tla)
                    elif node==nodeInComp[-1] and toNode==nodeInComp[0]:
                        # Check that repeated element is actually contiguous inside otherwise it is not a loop and should not be visible.
                        tla_update = list(set(tla).intersection(*[set(fromLinks.get(nodeInComp[i],{}).get(nodeInComp[i+1],[])) for i in range(len(nodeInComp)-1)]))
                        
                        if len(tla_update)>0:
                            #Loop link, can only happen when with left arrival and right departure (or vice versa).
                            complArrivals = compLinks.setdefault('lArrivals',{})

                            complArrivals.setdefault((component['last_bin'],component['first_bin'],'fr'),set()).update(tla)
                            comprDepartures.setdefault((component['last_bin'],component['first_bin'],'tl'),set()).update(tla)
                
                #to right arrivals
                if len(tra)>0: # Most probably the problem is here! Check it!
                    if toNode not in nodeInComp:
                        comprDepartures.setdefault((component['last_bin'],toLastComp['last_bin'],'tr'),set()).update(tra)
            
            #left departures
            if len(ld)>0 and doLeft:
                tla,tra = splitforwardInversedNodeComp(ld,toFirstComp,intermediateCondition)
                
                complDepartures = compLinks.setdefault('lDepartures',{})
                #to left arrivals
                if len(tla)>0:
                    if toNode not in nodeInComp:
                        complDepartures.setdefault((component['first_bin'],toFirstComp['first_bin'],'tl'),set()).update(tla)
                
                if len(tra)>0:
                    if toNode not in nodeInComp:
                        if component['first_bin']-1!=toFirstComp['last_bin']:
                            complDepartures.setdefault((component['first_bin'],toLastComp['last_bin'],'tr'),set()).update(tra)
                    elif node==nodeInComp[0] and toNode==nodeInComp[-1]:
                        # Check that repeated element is actually contiguous inside otherwise it is not a loop and should not be visible.
                        tla_update = list(set(tra).intersection(*[set(fromLinks.get(nodeInComp[i],{}).get(nodeInComp[i-1],[])) for i in range(len(nodeInComp)-1,0,-1)]))
                        
                        if len(tla_update)>0:
                            #Loop link, can only happen when with right arrival and left departure (or vice versa).
                            comprArrivals = compLinks.setdefault('rArrivals',{})

                            comprArrivals.setdefault((component['first_bin'],component['last_bin'],'fl'),set()).update(tra)
                            complDepartures.setdefault((component['first_bin'],component['last_bin'],'tr'),set()).update(tra)
                        
    return compLinks